In [23]:
!pip install pyLDAvis
import pyLDAvis.gensim_models

     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 MB 660.6 kB/s eta 0:00:04
     ---- ----------------------------------- 0.3/2.6 MB 3.8 MB/s eta 0:00:01
     ---------------- ----------------------- 1.1/2.6 MB 8.4 MB/s eta 0:00:01
     -------------------------------- ------- 2.1/2.6 MB 12.3 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 11.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
     ---------------------------------------- 67.1/67.1 kB ? eta 0:00:00
  Obtaining dependency information for simpful from https://files.pythonhosted.org/packages/8d/93/8448d3f1aa9d2911b8cba2602aaa1af85eb31a26d28b7b737f1fa5b40c02/simpful-2.11.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with 

In [24]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis.gensim_models

import matplotlib.pyplot as plt
import pandas as pd

In [25]:
import spacy
spacy.load("en_core_web_sm")
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [26]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\staylor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [28]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [29]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\staylor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [31]:
import csv
import pandas as pd 

In [32]:
text_data = []
with open("merge.csv") as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
print(tokens)
text_data.append(tokens)

['getting', 'classroom', 'excite', 'classroom', 'though', 'career', 'school', 'classroom', 'still', 'interest', 'observe']


In [33]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [34]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [35]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

### Try 5 topics

In [36]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [37]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.111*"getting" + 0.111*"school" + 0.111*"though" + 0.111*"still"')
(1, '0.250*"classroom" + 0.094*"interest" + 0.094*"career" + 0.094*"observe"')
(2, '0.111*"school" + 0.111*"excite" + 0.111*"though" + 0.111*"still"')
(3, '0.111*"getting" + 0.111*"school" + 0.111*"though" + 0.111*"excite"')
(4, '0.111*"getting" + 0.111*"school" + 0.111*"excite" + 0.111*"though"')


In [38]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[]
[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]


In [39]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.111*"though" + 0.111*"excite" + 0.111*"school" + 0.111*"interest"')
(1, '0.111*"interest" + 0.111*"though" + 0.111*"excite" + 0.111*"still"')
(2, '0.238*"classroom" + 0.095*"career" + 0.095*"getting" + 0.095*"observe"')


In [40]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.261*"classroom" + 0.092*"interest" + 0.092*"still" + 0.092*"school"')
(1, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(2, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(3, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(4, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(5, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(6, '0.111*"getting" + 0.111*"classroom" + 0.111*"school" + 0.111*"interest"')
(7, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')
(8, '0.111*"getting" + 0.111*"classroom" + 0.111*"school" + 0.111*"interest"')
(9, '0.111*"classroom" + 0.111*"interest" + 0.111*"still" + 0.111*"school"')


### pyLDAvis

In [41]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [51]:
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.enable_notebook(lda_display)

In [63]:
pyLDAvis.show(lda_display, ip='127.0.0.1', port=8888, n_retries=50, local=True, open_browser=True, http_server=None,)

OSError: [Errno 22] Invalid argument: 'https://cdn.jsdelivr.net/gh/bmabey/pyLDAvis@3.4.0/pyLDAvis/js/ldavis.v1.0.0.css'

In [47]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim_models.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

In [49]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim_models.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.enable_notebook(lda_display10)